In [1]:
import pandas as pd

# parquet 파일 경로 지정
file_path = 'multiarith.parquet'

# pyarrow(또는 fastparquet) 기반으로 parquet 파일 읽기
df = pd.read_parquet(file_path)

In [2]:
df.head()

,cleaning_status,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_strategy,input,output_program,output_answer,split,dataset
0,consensus,Solve the following math word problem.\n\n At ...,Solve the following math word problem.\n\n At ...,[15],[15],math,At the schools book fair Sam bought 13 advent...,answer=((13.0+17.0)-15.0)\nprint(answer),15,test,multiarith
1,consensus,Solve the following math word problem.\n\n Sam...,Solve the following math word problem.\n\n Sam...,[43],[43],math,Sam had 10 socks. If he threw away 3 old ones...,answer=((10.0+36.0)-3.0)\nprint(answer),43,test,multiarith
2,consensus,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...,[2],[2],math,For Halloween Megan received 11 pieces of can...,answer=((11.0+5.0)/8.0)\nprint(answer),2,test,multiarith
3,consensus,Solve the following math word problem.\n\n Deb...,Solve the following math word problem.\n\n Deb...,[2],[2],math,Debby's class is going on a field trip to the...,answer=((2.0+6.0)/4.0)\nprint(answer),2,test,multiarith
4,consensus,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...,[51],[51],math,For the school bake sale Amy made pastries. S...,answer=((15.0+48.0)-12.0)\nprint(answer),51,test,multiarith


In [3]:
df['platinum_prompt'][0]

'Solve the following math word problem.\n\n At the schools book fair Sam bought 13 adventure books and 17 mystery books. If 15 of the books were used, how many new books did he buy? \n\nThink step-by-step. Then, provide the final answer as a single number in the format "Answer: XXX" with no extra formatting.'

In [4]:
import pandas as pd
import re

# 1. Read the question again: ...
def repeat_question(match):
    # match.group(1): 첫 번째 줄바꿈 (예: "\n\n" 또는 "\n   \n")
    # match.group(2): 질문 텍스트
    # match.group(3): 두 번째 줄바꿈 (예: "\n\n")
    # 원하는 결과: 첫 번째 줄바꿈 뒤 기존 질문 대신 "Read the question again:" + 질문 텍스트를 두 번 반복하고,
    # 두 번째 줄바꿈을 그대로 붙인다.
    return f"{match.group(1)}{match.group(2)}\n\nRead the question again: {match.group(2)}{match.group(3)}"

# \n\n 사이에 공백이 있을 수도 있으므로 \n\s*\n 형태로 패턴을 수정
df['RE2'] = df['platinum_prompt'].str.replace(
    r'(\n\s*\n)(.*?)(\n\s*\n)(?=Think step-by-step)',
    repeat_question,
    flags=re.DOTALL,
    regex=True
)

# 2. Summarize the question first.
df['sum'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Summarize the question first. ",
    regex=True
)

# 3. Organize the question into a table first.
df['table'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Organize the question into a table first. ",
    regex=True
)

# 4. Organize the question into a graph structure first.
df['graph'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Organize the question into a graph structure first. ",
    regex=True
)

# 5. Summarize the question in bullet points first.
df['bullet_point'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Summarize the question in bullet points first. ",
    regex=True
)

# 6. Read the question again first.
df['sRE2'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Read the question again first. ",
    regex=True
)

In [15]:
df['RE2'][0]

'Solve the following math word problem.\n\n At the schools book fair Sam bought 13 adventure books and 17 mystery books. If 15 of the books were used, how many new books did he buy? \n\nRead the question again:  At the schools book fair Sam bought 13 adventure books and 17 mystery books. If 15 of the books were used, how many new books did he buy? \n\nThink step-by-step. Then, provide the final answer as a single number in the format "Answer: XXX" with no extra formatting.'

In [11]:
df['platinum_prompt_no_cot'][0]

'Solve the following math word problem.\n\n At the schools book fair Sam bought 13 adventure books and 17 mystery books. If 15 of the books were used, how many new books did he buy? \n\nThen, provide the final answer as a single number in the format "Answer: XXX" with no extra formatting.'

In [12]:
import pandas as pd
import re

def repeat_question(match):
    # match.group(1): 첫 번째 줄바꿈 (예: "\n\n" 또는 "\n   \n")
    # match.group(2): 질문 텍스트
    # match.group(3): 두 번째 줄바꿈 (예: "\n\n")
    # 원하는 결과: 첫 번째 줄바꿈 뒤 기존 질문 대신 "Read the question again:" + 질문 텍스트를 두 번 반복하고,
    # 두 번째 줄바꿈을 그대로 붙인다.
    return f"{match.group(1)}{match.group(2)}\n\nRead the question again: {match.group(2)}{match.group(3)}"

# \n\n 사이에 공백이 있을 수도 있으므로 \n\s*\n 형태로 패턴을 수정
df['RE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(\n\s*\n)(.*?)(\n\s*\n)(?=Then, provide the final answer as a single number)',
    repeat_question,
    flags=re.DOTALL,
    regex=True
)

df['sum_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer as a single number)',
    "Summarize the question first. ",
    regex=True
)

df['table_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer as a single number)',
    "Organize the question into a table first. ",
    regex=True
)

df['graph_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer as a single number)',
    "Organize the question into a graph structure first. ",
    regex=True
)

df['bullet_point_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer as a single number)',
    "Summarize the question in bullet points first. ",
    regex=True
)

df['sRE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer as a single number)',
    "Read the question again first. ",
    regex=True
)

In [20]:
df['sum_no_cot'][0]

'Solve the following math word problem.\n\n At the schools book fair Sam bought 13 adventure books and 17 mystery books. If 15 of the books were used, how many new books did he buy? \n\nSummarize the question first. Then, provide the final answer as a single number in the format "Answer: XXX" with no extra formatting.'

In [21]:
df.to_parquet('multiarith_add_task.parquet')

In [22]:
df.head()

,cleaning_status,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_strategy,input,output_program,output_answer,split,...,table,graph,bullet_point,sRE2,RE2_no_cot,sum_no_cot,table_no_cot,graph_no_cot,bullet_point_no_cot,sRE2_no_cot
0,consensus,Solve the following math word problem.\n\n At ...,Solve the following math word problem.\n\n At ...,[15],[15],math,At the schools book fair Sam bought 13 advent...,answer=((13.0+17.0)-15.0)\nprint(answer),15,test,...,Solve the following math word problem.\n\n At ...,Solve the following math word problem.\n\n At ...,Solve the following math word problem.\n\n At ...,Solve the following math word problem.\n\n At ...,Solve the following math word problem.\n\n At ...,Solve the following math word problem.\n\n At ...,Solve the following math word problem.\n\n At ...,Solve the following math word problem.\n\n At ...,Solve the following math word problem.\n\n At ...,Solve the following math word problem.\n\n At ...
1,consensus,Solve the following math word problem.\n\n Sam...,Solve the following math word problem.\n\n Sam...,[43],[43],math,Sam had 10 socks. If he threw away 3 old ones...,answer=((10.0+36.0)-3.0)\nprint(answer),43,test,...,Solve the following math word problem.\n\n Sam...,Solve the following math word problem.\n\n Sam...,Solve the following math word problem.\n\n Sam...,Solve the following math word problem.\n\n Sam...,Solve the following math word problem.\n\n Sam...,Solve the following math word problem.\n\n Sam...,Solve the following math word problem.\n\n Sam...,Solve the following math word problem.\n\n Sam...,Solve the following math word problem.\n\n Sam...,Solve the following math word problem.\n\n Sam...
2,consensus,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...,[2],[2],math,For Halloween Megan received 11 pieces of can...,answer=((11.0+5.0)/8.0)\nprint(answer),2,test,...,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...
3,consensus,Solve the following math word problem.\n\n Deb...,Solve the following math word problem.\n\n Deb...,[2],[2],math,Debby's class is going on a field trip to the...,answer=((2.0+6.0)/4.0)\nprint(answer),2,test,...,Solve the following math word problem.\n\n Deb...,Solve the following math word problem.\n\n Deb...,Solve the following math word problem.\n\n Deb...,Solve the following math word problem.\n\n Deb...,Solve the following math word problem.\n\n Deb...,Solve the following math word problem.\n\n Deb...,Solve the following math word problem.\n\n Deb...,Solve the following math word problem.\n\n Deb...,Solve the following math word problem.\n\n Deb...,Solve the following math word problem.\n\n Deb...
4,consensus,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...,[51],[51],math,For the school bake sale Amy made pastries. S...,answer=((15.0+48.0)-12.0)\nprint(answer),51,test,...,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...,Solve the following math word problem.\n\n For...
